In [1]:
import json
from typing import List, Dict

def prepare_finetuning_data(raw_conversations: List[Dict]) -> List[Dict]:
    finetuning_data = []
    
    for conversation in raw_conversations:
        context = ""
        for turn in conversation:
            if turn["role"] == "user":
                # Chuẩn bị prompt
                prompt = f"{context}User: {turn['content']}\nAssistant: "
                
                # Tìm phản hồi tương ứng của assistant
                assistant_response = next((t["content"] for t in conversation if t["role"] == "assistant" and t["turn_id"] > turn["turn_id"]), None)
                
                if assistant_response:
                    finetuning_data.append({
                        "prompt": prompt,
                        "response": assistant_response,
                        "intent": turn.get("intent", ""),  # Nếu có nhãn intent
                        "entities": turn.get("entities", [])  # Nếu có nhãn entities
                    })
                
                context += f"User: {turn['content']}\nAssistant: {assistant_response}\n"
            
    return finetuning_data

# Giả sử bạn có dữ liệu hội thoại thô
raw_conversations = [
    [
        {"role": "user", "turn_id": 1, "content": "Tôi muốn xem một bộ phim hành động.", "intent": "request_movie_recommendation", "entities": ["hành động"]},
        {"role": "assistant", "turn_id": 2, "content": "Tôi hiểu bạn đang tìm kiếm một bộ phim hành động. Bạn có yêu cầu cụ thể nào về diễn viên hoặc năm phát hành không?"},
        {"role": "user", "turn_id": 3, "content": "Tôi thích phim của Tom Cruise.", "intent": "specify_preference", "entities": ["Tom Cruise"]},
        {"role": "assistant", "turn_id": 4, "content": "Tuyệt vời! Tôi đề xuất bạn xem 'Mission: Impossible - Fallout'. Đây là một bộ phim hành động xuất sắc với Tom Cruise trong vai chính. Bạn có muốn biết thêm về bộ phim này không?"}
    ]
    # Thêm các cuộc hội thoại khác vào đây
]

# Chuẩn bị dữ liệu fine-tuning
finetuning_data = prepare_finetuning_data(raw_conversations)

# Lưu dữ liệu fine-tuning vào file
with open("qwen_finetuning_data.json", "w", encoding="utf-8") as f:
    json.dump(finetuning_data, f, ensure_ascii=False, indent=2)

print(f"Đã tạo {len(finetuning_data)} mẫu dữ liệu fine-tuning.")

Đã tạo 2 mẫu dữ liệu fine-tuning.


In [13]:
import json

with open('/home/thiendc/InferConverRec/tempt/test_data_dbpedia.jsonl', 'r') as f:
    datas= f.readlines()
json.loads(datas[1]).get('messages')

[{'timeOffset': 0,
  'text': 'Hello!',
  'senderWorkerId': 958,
  'messageId': 204187,
  'entity': [],
  'entity_name': [],
  'movie': [],
  'movie_name': [],
  'word_name': []},
 {'timeOffset': 8,
  'text': 'Hello!',
  'senderWorkerId': 959,
  'messageId': 204188,
  'entity': [],
  'entity_name': [],
  'movie': [],
  'movie_name': [],
  'word_name': []},
 {'timeOffset': 30,
  'text': 'What kind of movies do you like?',
  'senderWorkerId': 958,
  'messageId': 204189,
  'entity': [],
  'entity_name': [],
  'movie': [],
  'movie_name': [],
  'word_name': ['movies']},
 {'timeOffset': 60,
  'text': 'I am looking for a movie recommendation.   When I was younger I really enjoyed the @77161',
  'senderWorkerId': 959,
  'messageId': 204190,
  'entity': [],
  'entity_name': [],
  'movie': ['<http://dbpedia.org/resource/A_Nightmare_on_Elm_Street>'],
  'movie_name': ['A Nightmare on Elm Street (1984)'],
  'word_name': ['recommendation',
   'movie',
   'enjoyed',
   'nightmare',
   'elm',
   'stre

In [1]:
import re

movies = {
    "111776": "Super Troopers (2001)",
    "91481": "Beverly Hills Cop (1984)",
    "151656": "Police Academy (1984)",
    "134643": "American Pie (1999)",
    "192131": "American Pie",
    "124771": "48 Hrs. (1982)",
    "94688": "Police Academy 2: Their First Assignment (1985)",
    "101794": "Lethal Weapon (1987)",
    "204334": "Happy Death Day (2017)",
    "125431": "Annabelle (2014)",
    "118338": "The Forest (2016)",
    "205430": "A Quiet Place (2018)",
    "132562": "The Last House on the Left (1972)",
    "157765": "The Heat (2013)",
    "187061": "Identity Thief (2013)",
}

# Chuẩn hóa tên phim
def normalize_movie_title(title):
    # Xóa năm trong ngoặc đơn và loại bỏ khoảng trắng thừa
    title = re.sub(r'\s*\(\d{4}\)', '', title).strip()
    return title.lower()

# Tìm kiếm phim
def find_movie(user_input):
    normalized_input = normalize_movie_title(user_input)
    for movie_id, movie_title in movies.items():
        if normalize_movie_title(movie_title) == normalized_input:
            return movie_id, movie_title
    return None

# Ví dụ tìm kiếm
user_input = "The Forest"
result = find_movie(user_input)

if result:
    print(f"Phim bạn tìm là: {result[1]} (ID: {result[0]})")
else:
    print("Không tìm thấy phim.")

Phim bạn tìm là: The Forest (2016) (ID: 118338)
